<a href="https://colab.research.google.com/github/rashaKM2025/Data-Science-Salary-Prediction/blob/main/gld_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
df = pd.read_csv('/content/drive/MyDrive/glassdoor/glassdoorjobs_cleaning.csv')
df

,unnamed:_0,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,min_salary,max_salary,company_name_clean,avg_salary
0,0,Data Scientist,$53K-$91K (Glassdoor est.),Job Description,3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,NaN,NaN,Tecolote Research\n,NaN
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),Job Description,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,NaN,NaN,University of Maryland Medical System\n,NaN
2,2,Data Scientist,$80K-$90K (Glassdoor est.),Job Description,4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,NaN,NaN,KnowBe,NaN
3,3,Data Scientist,$56K-$97K (Glassdoor est.),Job Description,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",NaN,NaN,PNNL\n,NaN
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Job Description,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",NaN,NaN,Affinity Solutions\n,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,922,Data Scientist,-1,Job Description,4.7,"Steel Point Solutions, LLC\n4.7","Alexandria, VA","Calverton, MD",1 to 50 employees,2013,Other Organization,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1,NaN,NaN,Steel Point Solutions,NaN
593,925,Data Scientists,-1,Job Description,3.3,Miracle Software Systems\n3.3,Michigan,"Novi, MI",51 to 200 employees,1994,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),"CSC, IBM",NaN,NaN,Miracle Software Systems\n,NaN
594,928,"Associate Scientist / Sr. Associate Scientist,...",$59K-$125K (Glassdoor est.),Job Description,4.0,23andMe\n4.0,"South San Francisco, CA","Sunnyvale, CA",501 to 1000 employees,2006,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,"Ancestry, Verily Life Sciences, Abbott Laborat...",NaN,NaN,andMe\n,NaN
595,945,Machine Learning Engineer (NLP),$80K-$142K (Glassdoor est.),Job Description,4.1,CK-12 Foundation\n4.1,"Palo Alto, CA","Palo Alto, CA",1 to 50 employees,2007,Company - Private,K-12 Education,Education,Unknown / Non-Applicable,-1,NaN,NaN,CK,NaN


In [39]:
text_cols = ['job_title','company_name','company_name_clean','location',
             'headquarters','industry','sector','type_of_ownership','revenue']
for col in text_cols:
  if col in df.columns:
    df[col] = df[col].astype(str).str.lower().str.strip()

In [40]:
df['min_salary'] = pd.to_numeric(df['min_salary'], errors='coerce')
df['max_salary'] = pd.to_numeric(df['max_salary'], errors='coerce')
df['avg_salary'] = pd.to_numeric(df['avg_salary'], errors='coerce')

In [41]:
df[['min_salary','max_salary','avg_salary']] = df[['min_salary','max_salary','avg_salary']] / 1000

In [42]:
scaler = MinMaxScaler()
df[['min_salary_norm','max_salary_norm','avg_salary_norm']] = scaler.fit_transform(
    df[['min_salary','max_salary','avg_salary']].fillna(0)
)

In [43]:
categorical_cols = ['type_of_ownership','industry','sector']
for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col+'_encoded'] = le.fit_transform(df[col].astype(str))

In [44]:
df['salary_range'] = df['max_salary'] - df['min_salary']

In [47]:
df['company_name_clean'] = (
    df['company_name']
    .astype(str)
    .str.replace(r'\n\d+\.\d+', '', regex=True)
    .str.replace(r'\d+\.\d+', '', regex=True)
    .str.replace(r'\n', ' ', regex=True)
    .str.strip()
)

In [49]:
df[['city','state']] = df['location'].str.split(',', n=1, expand=True)

df['city'] = df['city'].str.strip().str.lower()
df['state'] = df['state'].str.strip().str.lower()

In [50]:
size_map = {
    '1 to 50 employees': 1,
    '51 to 200 employees': 2,
    '201 to 500 employees': 3,
    '501 to 1000 employees': 4,
    '1001 to 5000 employees': 5,
    '5001 to 10000 employees': 6,
    '10000+ employees': 7
}
df['company_size_encoded'] = df['size'].map(size_map)

In [51]:
revenue_map = {
    'less than $1 million (usd)': 0.5,
    '$1 to $5 million (usd)': 3,
    '$5 to $10 million (usd)': 7.5,
    '$10 to $25 million (usd)': 17.5,
    '$25 to $50 million (usd)': 37.5,
    '$50 to $100 million (usd)': 75,
    '$100 to $500 million (usd)': 300,
    '$500 million to $1 billion (usd)': 750,
    '$1 to $2 billion (usd)': 1500,
    '$2 to $5 billion (usd)': 3500,
    '$5 to $10 billion (usd)': 7500,
    '$10+ billion (usd)': 10000
}
df['revenue_million'] = df['revenue'].map(revenue_map)

In [55]:
df['company_age'] = 2026 - pd.to_numeric(df['founded'], errors='coerce')

In [56]:
def simplify_title(title):
    title = str(title).lower()
    if "scientist" in title:
        return "scientist"
    elif "analyst" in title:
        return "analyst"
    elif "engineer" in title:
        return "engineer"
    elif "manager" in title:
        return "manager"
    else:
        return "other"

df['job_title_simplified'] = df['job_title'].apply(simplify_title)

In [57]:
df['state'] = df['state'].fillna("unknown")
df['revenue_million'] = df['revenue_million'].fillna(0)
df['company_size_encoded'] = df['company_size_encoded'].fillna(0)
df['company_age'] = df['company_age'].fillna(0)

In [52]:
df.head()

,unnamed:_0,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,...,max_salary_norm,avg_salary_norm,type_of_ownership_encoded,industry_encoded,sector_encoded,salary_range,city,state,company_size_encoded,revenue_million
0,0,data scientist,$53K-$91K (Glassdoor est.),Job Description,3.8,tecolote research\n3.8,"albuquerque, nm","goleta, ca",501 to 1000 employees,1973,...,0.0,0.0,2,3,2,NaN,albuquerque,nm,4.0,75.0
1,1,healthcare data scientist,$63K-$112K (Glassdoor est.),Job Description,3.4,university of maryland medical system\n3.4,"linthicum, md","baltimore, md",10000+ employees,1984,...,0.0,0.0,8,28,12,NaN,linthicum,md,7.0,3500.0
2,2,data scientist,$80K-$90K (Glassdoor est.),Job Description,4.8,knowbe4\n4.8,"clearwater, fl","clearwater, fl",501 to 1000 employees,2010,...,0.0,0.0,2,48,6,NaN,clearwater,fl,4.0,300.0
3,3,data scientist,$56K-$97K (Glassdoor est.),Job Description,3.8,pnnl\n3.8,"richland, wa","richland, wa",1001 to 5000 employees,1965,...,0.0,0.0,5,18,19,NaN,richland,wa,5.0,750.0
4,4,data scientist,$86K-$143K (Glassdoor est.),Job Description,2.9,affinity solutions\n2.9,"new york, ny","new york, ny",51 to 200 employees,1998,...,0.0,0.0,2,2,6,NaN,new york,ny,2.0,NaN


In [58]:
df.to_csv('/content/glassdoorjobs_normalization.csv', index=False)


In [59]:
from google.colab import files
files.download('glassdoorjobs_normalization.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>